# Data Preprocessing 

This notebook performs ETL and preprocessing for NYC TLC trip data (yellow taxis and FHVs) and corresponding weather data.  
It standardizes column names, aligns datatypes, handles missing values, and writes cleaned Parquet files ready for analysis.

## 1. Download files

In [27]:
# import library for downloading data
import os
from urllib.request import urlretrieve
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
import pyarrow.parquet as pq
from pyspark.ml.feature import Imputer

In [ ]:
output_relative_dir = 'data/'
vehicles_dir = 'data/tlc_data/'

# check if it file exists as makedir will raise an error if it does exist
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
# for each type of data set we will need, we will create the paths
for target_dir in ('tlc_data', 'taxi_zones'): 
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)
        
for target_dir in ('yellow','hvfhv'): 
    if not os.path.exists(vehicles_dir + target_dir):
        os.makedirs(vehicles_dir + target_dir);

In [ ]:
YEAR = "2024"
MONTHS = range(1,7)
# URL TEMPLATES
yellow_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_"
hvfhv_url =  "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_"

In [ ]:
tlc_output_dir = '../data/tlc_data/'

for file_dir, base_url in zip(
    ('yellow', 'hvfhv'),
    (yellow_url, hvfhv_url)
):
    final_path = f"{tlc_output_dir}{file_dir}"
    
    for month in MONTHS:
        month_str = str(month).zfill(2) 
        
        print(f"Begin month {month_str} for {file_dir}")
        
        current_url = f"{base_url}{YEAR}-{month_str}.parquet"
        output_dir = f"{final_path}/{YEAR}-{month_str}.parquet"
        
        # Download file
        urlretrieve(current_url, output_dir)
        
        print(f"Completed month {month_str} for {file_dir}")

Begin month 01 for yellow
Completed month 01 for yellow
Begin month 02 for yellow
Completed month 02 for yellow
Begin month 03 for yellow
Completed month 03 for yellow
Begin month 04 for yellow
Completed month 04 for yellow
Begin month 05 for yellow
Completed month 05 for yellow
Begin month 06 for yellow
Completed month 06 for yellow
Begin month 01 for hvfhv
Completed month 01 for hvfhv
Begin month 02 for hvfhv
Completed month 02 for hvfhv
Begin month 03 for hvfhv
Completed month 03 for hvfhv
Begin month 04 for hvfhv
Completed month 04 for hvfhv
Begin month 05 for hvfhv
Completed month 05 for hvfhv
Begin month 06 for hvfhv
Completed month 06 for hvfhv


## 2. Initial Cleaning

In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("TLC eda")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

### a. Yellow

In [ ]:
# Load yellow taxi sample
sdf_yellow_example = spark.read.parquet('../data/tlc_data/yellow/2024-01.parquet')
sdf_yellow_example.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [ ]:
# Fix naming and datatypes for one example and get the schema 
sdf_yellow_example = (sdf_yellow_example.withColumn('passenger_count', F.col('passenger_count').cast('int')).withColumn('RatecodeID', F.col('RatecodeID').cast('int')).withColumn('payment_type', F.col('payment_type').cast('int')).withColumn('store_and_fwd_flag', (F.col("store_and_fwd_flag") == 'Y').cast('boolean')))
consistent_col_casing_yellow = [F.col(col_name).alias(col_name.lower()) for col_name in sdf_yellow_example.columns]
sdf_yellow_example = sdf_yellow_example.select(*consistent_col_casing_yellow)
sdf_yellow_example = (sdf_yellow_example.withColumnRenamed("vendorid", "vendor_id").withColumnRenamed("ratecodeid", "ratecode_id").withColumnRenamed("pulocationid", "pu_location_id").withColumnRenamed("dolocationid", "do_location_id").withColumnRenamed("trip_distance", "trip_miles").withColumnRenamed("tpep_pickup_datetime", "pickup_datetime").withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime"))
sdf_schema_yellow = sdf_yellow_example.schema
sdf_schema_yellow

In [68]:
YEAR = "2024"  
MONTHS = range(1, 7)  
TLC_OUTPUT_DIR = '../data/tlc_data/'


def align_datatypes_yellow(path, schema, year, month):
    df = spark.read.parquet(path)
    # fix column names
    df = df.select(*consistent_col_casing_yellow)
    df = (df.withColumnRenamed("vendorid", "vendor_id").withColumnRenamed("ratecodeid", "ratecode_id").withColumnRenamed("pulocationid", "pu_location_id").withColumnRenamed("dolocationid", "do_location_id").withColumnRenamed("trip_distance", "trip_miles").withColumnRenamed("tpep_pickup_datetime", "pickup_datetime").withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime"))
    # Cast every column to match the saved schema's data types
    for field in schema:
        if field.name in df.columns:  
            df = df.withColumn(field.name, F.col(field.name).cast(field.dataType))
            
    output_path = f'{TLC_OUTPUT_DIR}raw/yellow/{year}-{str(month).zfill(2)}'
    df.coalesce(1).write.mode('overwrite').parquet(output_path)
    return None

In [69]:
for month in MONTHS:
    input_path = f'{TLC_OUTPUT_DIR}yellow/{YEAR}-{str(month).zfill(2)}.parquet'
    align_datatypes_yellow(input_path, sdf_schema_yellow, YEAR, month)

### b. HVFHV data

In [49]:
sdf_fhv_example = spark.read.parquet('../data/tlc_data/hvfhv/2024-01.parquet')
sdf_fhv_example.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- on_scene_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_

In [ ]:
sdf_fhv_example = (sdf_fhv_example.withColumn('trip_time', F.col('trip_time').cast('double')).withColumn('shared_request_flag', (F.col("shared_request_flag") == 'Y').cast('boolean')).withColumn('shared_match_flag', (F.col("shared_match_flag") == 'Y').cast('boolean')).withColumn('access_a_ride_flag', (F.col("access_a_ride_flag") == 'Y').cast('boolean')).withColumn('wav_request_flag', (F.col("wav_request_flag") == 'Y').cast('boolean')).withColumn('wav_match_flag', (F.col("wav_match_flag") == 'Y').cast('boolean')))
consistent_col_casing_fhv = [F.col(col_name).alias(col_name.lower()) for col_name in sdf_fhv_example.columns]
sdf_schema_fhv = sdf_fhv_example.schema
sdf_schema_fhv

In [15]:
YEAR = "2024"  
MONTHS = range(6,7)  
TLC_OUTPUT_DIR = '../data/tlc_data/'

def align_datatypes_hvfhv(path, schema, year, month):
    df = spark.read.parquet(path)
    # fix column names
    df = df.select(*consistent_col_casing_fhv)
    df = (df.withColumnRenamed("pulocationID", "pu_location_id").withColumnRenamed("dolocationid", "do_location_id").withColumnRenamed("base_passenger_fare", "fare_amount"))
    # Cast every column to match the saved schema's data types
    for field in schema:
        if field.name in df.columns:  
            df = df.withColumn(field.name, F.col(field.name).cast(field.dataType))
            
    
    output_path = f'{TLC_OUTPUT_DIR}raw/hvfhv/{year}-{str(month).zfill(2)}'
    df.coalesce(1).write.mode('overwrite').parquet(output_path)
    return None

In [16]:
for month in MONTHS:
    input_path = f'{TLC_OUTPUT_DIR}hvfhv/{YEAR}-{str(month).zfill(2)}.parquet'
    align_datatypes_hvfhv(input_path, sdf_schema_fhv, YEAR, month)

In [57]:
def get_shape(service_type, year, months, base_dir="../data/tlc_data"):
    """
    Summarize row counts and features for a given service type and year.

    Parameters:
        service_type 
        year 
        months 
        base_dir 
    Returns:
        dict with summary
    """
    total_rows = 0
    example_path = f"{base_dir}/{service_type}/{year}-{str(months[0]).zfill(2)}.parquet"
    example_file = pq.ParquetFile(example_path)
    num_features = example_file.metadata.num_columns

    for month in months:
        path = f"{base_dir}/{service_type}/{year}-{str(month).zfill(2)}.parquet"
        parquet_file = pq.ParquetFile(path)
        total_rows += parquet_file.metadata.num_rows

    summary = {
        "service_type": service_type,
        "year": year,
        "months": f"{months[0]}–{months[-1]}",
        "rows": total_rows,
        "features": num_features,
    }
    return summary



hvfhv_2024 = get_shape("hvfhv", 2024, range(1, 7))
hvfhv_2025 = get_shape("hvfhv", 2025, range(1, 7))
yellow_2024 = get_shape("yellow", 2024, range(1, 7))
yellow_2025 = get_shape("yellow", 2025, range(1, 7))

for s in [yellow_2024, yellow_2025, hvfhv_2024, hvfhv_2025]:
    print(f"{s['service_type'].upper()} {s['year']}")
    print(f"  Total rows: {s['rows']:,}")
    print(f"  Features: {s['features']}")
    print("-" * 40)


YELLOW 2024
  Total rows: 20,332,093
  Features: 19
----------------------------------------
YELLOW 2025
  Total rows: 24,083,384
  Features: 20
----------------------------------------
HVFHV 2024
  Total rows: 120,864,668
  Features: 24
----------------------------------------
HVFHV 2025
  Total rows: 120,995,191
  Features: 25
----------------------------------------


25/09/02 05:45:40 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 908013 ms exceeds timeout 120000 ms
25/09/02 05:45:40 WARN SparkContext: Killing executors is not supported by current scheduler.
25/09/02 05:45:42 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

### c. Weather data

In [28]:
df_weather_24 = spark.read.option("header", "true").option("sep", "|").csv('/Users/tasneemzulaiqa/Documents/GitHub/project-1-individual-tasneemzulaiqa/data/weather_data/2024.psv')            
df_weather_25 = spark.read.option("header", "true").option("sep", "|").csv('/Users/tasneemzulaiqa/Documents/GitHub/project-1-individual-tasneemzulaiqa/data/weather_data/GHCNh_USW00094789_2025.psv')
df_weather_25.printSchema()               

root
 |-- STATION: string (nullable = true)
 |-- Station_name: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- Elevation: string (nullable = true)
 |-- temperature: string (nullable = true)
 |-- temperature_Measurement_Code: string (nullable = true)
 |-- temperature_Quality_Code: string (nullable = true)
 |-- temperature_Report_Type: string (nullable = true)
 |-- temperature_Source_Code: string (nullable = true)
 |-- temperature_Source_Station_ID: string (nullable = true)
 |-- dew_point_temperature: string (nullable = true)
 |-- dew_point_temperature_Measurement_Code: string (nullable = true)
 |-- dew_point_temperature_Quality_Code: string (nullable = true)
 |-- dew_point_temperature_Report_Type: string (nullable = true)
 |-- dew_point_temperature_Source_Code: string (nullable = true)
 |-- dew_point_temperature_Source_Station_ID: string (nullable = true)
 |-- station_level_pressure: strin

In [ ]:
# Keep only relevant dates
df_weather_24 = df_weather_24.filter((F.month(F.col("DATE")) >= 1) & (F.month(F.col("DATE")) <= 6))
df_weather_25 = df_weather_25.filter((F.month(F.col("DATE")) >= 1) & (F.month(F.col("DATE")) <= 6))

In [33]:
# Shape
nweather_24 = df_weather_24.count()
nfeat_weather_24 = len(df_weather_24.columns)

nweather_25 = df_weather_25.count()
nfeat_weather_25 = len(df_weather_25.columns)

print("WEATHER SHAPE")
print("Number of rows weather 2024:", nweather_24)
print("Number of features weather 2024:", nfeat_weather_24)
print("Number of rows weather 2025:", nweather_25)
print("Number of features weather 2025:", nfeat_weather_25)

WEATHER SHAPE
Number of rows weather 2024: 5672
Number of features weather 2024: 234
Number of rows weather 2025: 3310
Number of features weather 2025: 234


In [34]:
df_weather_24 = df_weather_24.select("Station_name", "DATE", "temperature", "wind_speed", "precipitation")
df_weather_25 = df_weather_25.select("Station_name", "DATE", "temperature", "wind_speed", "precipitation")

In [35]:
# Fix datatypes and naming
df_weather_24 = (df_weather_24.withColumn('DATE', F.col('DATE').cast('timestamp')).withColumn('temperature', F.col('temperature').cast('double')).withColumn('wind_speed', F.col('wind_speed').cast('double')).withColumn('precipitation', F.col('precipitation').cast('double')))
df_weather_25 = (df_weather_25.withColumn('DATE', F.col('DATE').cast('timestamp')).withColumn('temperature', F.col('temperature').cast('double')).withColumn('wind_speed', F.col('wind_speed').cast('double')).withColumn('precipitation', F.col('precipitation').cast('double')))
df_weather_24 = (df_weather_24.withColumnRenamed("DATE", "date_time").withColumnRenamed('Station_name','station_name'))
df_weather_25 = (df_weather_25.withColumnRenamed("DATE", "date_time").withColumnRenamed('Station_name','station_name'))

In [36]:
null_wind_speed_24 = df_weather_24.filter(F.col('wind_speed').isNull()).count()
null_temp_24 = df_weather_24.filter(F.col('temperature').isNull()).count()
null_precip_24 = df_weather_24.filter(F.col('precipitation').isNull()).count()

print("NULL COUNTS 2024")
print("Wind Speed 2024:", null_wind_speed_24)
print("Temperature 2024:", null_temp_24)
print("Precipitation 2024:", null_precip_24)

NULL COUNTS 2024
Wind Speed 2024: 536
Temperature 2024: 2
Precipitation 2024: 572


In [37]:
null_wind_speed_25 = df_weather_25.filter(F.col('wind_speed').isNull()).count()
null_temp_25 = df_weather_25.filter(F.col('temperature').isNull()).count()
null_precip_25 = df_weather_25.filter(F.col('precipitation').isNull()).count()

print("NULL COUNTS 2025")
print("Wind Speed 2025:", null_wind_speed_25)
print("Temperature 2025:", null_temp_25)
print("Precipitation 2025:", null_precip_25)

NULL COUNTS 2025
Wind Speed 2025: 0
Temperature 2025: 0
Precipitation 2025: 1022


In [38]:
# Replace precipitation nulls with 0
df_weather_24 = df_weather_24.fillna({'precipitation': 0})
df_weather_25 = df_weather_25.fillna({'precipitation': 0})

In [ ]:
df_weather_25.select('wind_speed').describe().show()


+-------+-----------------+
|summary|       wind_speed|
+-------+-----------------+
|  count|             3310|
|   mean|6.047734138972836|
| stddev| 2.93056330269313|
|    min|              0.0|
|    max|             16.0|
+-------+-----------------+



In [40]:
# impute wind speed with median
imputer = Imputer(inputCols=["wind_speed"],outputCols=["wind_speed"],strategy="median")
df_weather_24 = imputer.fit(df_weather_24).transform(df_weather_24)

In [41]:
# Shape
nweather_24 = df_weather_24.count()
nfeat_weather_24 = len(df_weather_24.columns)

nweather_25 = df_weather_25.count()
nfeat_weather_25 = len(df_weather_25.columns)

print("WEATHER SHAPE")
print("Number of rows weather 2024:", nweather_24)
print("Number of features weather 2024:", nfeat_weather_24)
print("Number of rows weather 2025:", nweather_25)
print("Number of features weather 2025:", nfeat_weather_25)

WEATHER SHAPE
Number of rows weather 2024: 5672
Number of features weather 2024: 5
Number of rows weather 2025: 3310
Number of features weather 2025: 5


In [44]:
output_path = "../data/tlc_data/raw/cleaned/curated/weather/2024"
df_weather_24.write.mode("overwrite").parquet(output_path)

In [ ]:
output_path = "../data/tlc_data/raw/cleaned/curated/weather/2025"
df_weather_25.write.mode("overwrite").parquet(output_path)